# Lab 3: Open-Loop Testing and System ID for a DC Motor

## Introduction



In this lab, you will use the `wxbd_gui` approach to perform an open-loop test on a DC motor.  You will then find a transfer function for the DC motor system and curve fit simulation results to the experimental data.

In order to find the TF for the DC motor system, you will run simulations for different simple transfer functions (building blocks) and think about how to assemble those simpler terms into a complete transfer function.

### Main Steps

- switch to an Arduino that is connected to a DC motor
- remove the beam from the motor by loosening the set screws
- create a block diagram model for your system
    - your plant with use an `h_bridge` as the actuator and an `encoder` as the sensor
- run a step response with your DC motor system
     - copy and paste the data from the serial monitor to a csv file
- run a pulse response with your DC motor system
    - replace the step input with a pulse input
        - more details below
    - again, copy data into a csv file
- simulate the step, impulse, and pulse responses for the three simple TFs, also called "building blocks" 
- think about how to combined the building blocks into a TF that captures the pulse response of the DC motor
- curve fit the pulse response of the DC motor to estimate any unknown coefficients in your TF
- use your curve fit results to create an estimated or optimized TF for the DC motor system
- use your estimated TF to predict the step response and compare that to the step response data 
    - you are not redoing the curve fit here, you are just trying to see if the estimated TF can also accurately predict the step response

## Learning Objectives

Students will learn

- how to assemble a TF from fundamental buildling blocks
- how to recognize the presence of building block terms in a step or pulse response test
- how to apply the `wxbd_gui` approach to a DC motor/encoder/H-bridge system 
- go deeper in their understanding of `optimize.fmin`

<a id="section_sw_update"></a>

## Transfer Function Fundamentals

A transfer function is an input/output model of a dynamic system. The transfer function represents the underlying differential equation that relates the input and output for the dynamic system.  A transfer function is a fraction with a polynomial of s in the numerator and denominator.  For example:

$$G(s) = \frac{3s+2}{s^2+7s+15}$$

And in general:

$$G(s) = \frac{b_m s^m + b_{m-1} s^{m-1} + \cdots + b_1 s + b_0}{s^n + a_{n-1} s^{n-1} + \cdots + a_1 s + a_0}$$

Any polynomial that has real coefficients can be factored into a product of first and second order terms.  So, the numerator and denominator of a transfer function can be thought of as products of such terms.  The roots of the denominator polynomial are referred to as poles and the roots of the numerator polynomial are referred to as zeros.  So I transfer function is made up of some combination of first or second order poles and potentially first or second order zeros.  Additionally, a pole at the origin (1/s) is referred to as a pure integrator and a zero at the origin (s) is referred to as a pure differentiator.

So, there are essentially six building blocks that an engineer can use to assemble a transfer function:

- first order pole
- second order pole
- pure integrator
- first order zero
- second order zero
- pure differentiator

Since we are just getting started, we will only focus on poles and pure integrators in this week's lab.

If an engineer is trying to infer the form of a transfer function based on experimental data, it is best to recognize which of these six building blocks needs to be included in the transfer function and then multiply those terms together.  In order to develop this skill, it is important for students to know how each of the building blocks responds to various inputs.






## Block Diagram Model

<img src="figs/OL_BD.jpeg" width=300px>

Conceptually, the block diagram shown above can represent any open-loop system: it could be a circuit, a car, a biomedical device, an autonomous vehicle, ... anything.

Last week, our plant (or physical system) was an RC circuit connected to an Arduino.  The Arduino was programmed to send $V_{in}$ to the circuit using `analogWrite` and to read a $V_{out}$ from the circuit.  Your block diagram this week will *look* exactly the same as last week, but the actuator and sensor will be different inside the plant.

**Note:** Your block diagram will not have an output node.  The output of every block is automatically printed to the Arduino serial monitor.

### `wxbd_gui` Step Response

In order to perform a step response test using the DC motor, create a step input block and a plant.  The only difference is that the plant this week uses a different actuator and a different sensor.  You may rememember for EGR 107/185 that an H-Bridge helps an Arduino drive a DC motor.  You may not have worked with an encoder before, but it is a sensor that measures the rotational position of a shaft.  So, when you add your plant block, choose `h_bridge` as the actuator and `encoder` as the sensor.

Set the step response as the input to the plant.  Then check the template path, set the output path, and generate the code.  Once you have generated the code, compile it and upload it to the Arduino and you are ready to run a test if your batteries are connected.

Qualitatively, what do you notice about the step response of the DC motor?  How do you observations compare to the data?

## OL Pulse Tests

A step is a signal that turns on and stays on.  An impulse is a signal that turns on and off very quickly.  A finite-width pulse (or just pulse for short) is inbetween the two: it switches on for a certain width of time and then switches back off:

<img src="figs/OL_pulse_input.png" width=500px>

The full name for the pulse is a finite width pulse.  This name is in contrast to an impulse, which is infintesimally narrow.  It would be very difficult and might not be super helpful to try and send an impulse voltage to the DC motor.  So, we will use a finite width pulse instead.  

### `wxbd_gui` Pulse Test

In order to perform a pulse test on the DC motor system, you just need to replace the step input block with a pulse input block.  To do this, choose "Replace Block" from the "Block" menu on the gui.  When the "Replace Block" dialog pops up, chose the step input U for the "Block to Replace".  Then specify a pulse input as the block you want to replace it with.

Run the test, save the data to a csv file, and plot it in a Jupyter Notebook.

## Simulating the Impulse, Step, and Pulse Responses for the TF Building Blocks

Your ultimate goal is to curve fit a transfer function to the pulse response data.  Before you can do this, you need to know what terms to include in the TF (this is sometimes referred to as finding the form of the TF).  A TF is built by combining building blocks together, as discussed above.  So, you need to know how each of the building blocks responds to various standard inputs: an impulse, a step, and a finite width pulse.

#### Building Block TFs

The building block transfer functions you are working with in this week's lab are

Pure integrator:

$$G_0(s) = \frac{1}{s}$$

First order pole:

$$G_1(s) = \frac{p}{s+p}$$

Underdamped, Second-Order Pole:

$$G_2(s) = \frac{\omega_n^2}{s^2 + 2 \zeta \omega_n s + \omega_n^2}$$

Use the numeric parameters $p = 5$, $\zeta = 0.1$, and $\omega_n = 2 \pi$.


For each of these building blocks, use the Python `control` module to find the impulse response, the step response, and the pulse response (use `control.forced_response` to find the pulase response).

## Assembling your TF

Study the simulation results for the building blocks and find a combination that leads to a pulse response that qualitatively matches the experimental pulse response.  When we want to combine building block terms, multiply them together. 

## Curve Fitting

Once you have a form for your TF, write a `mymodel` function that finds the pulse response for a given set of coefficients.  Verify that your `mymodel` function leads to a decent pulse response before you optimize using `optimize.fmin`.

## Verification Using the Step Response Data

Once you have completed the curve fit, use your optimized coefficents to create the "best" TF.  Use this best TF to predict the step response and compare this step response simulation to the step response data collected at the start of the lab.

## Comprehension Questions

1. How does a DC motor repond to an open-loop step input?  What does this response tell us about the TF for a DC motor and the building block(s) needed to model a DC motor?

2. Explain why each term in your final TF was included based on how the building block terms respond to different inputs.

3. Based on the pulse response, why *must* there be a pure integrator in the TF?

4. Again, based on the pulse response, why can the TF not be *just* a pure integrator? 